In [ ]:
import pandas as pd 
import numpy as np

import os

from tqdm import tqdm

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [ ]:
def read_table(table_path):
        df = pd.read_excel(table_path, skiprows=28)
        df = df.iloc[: , 8:]

        df.drop(df.tail(49).index,
                inplace = True)

        df = df.set_index('INCOME STATEMENT').transpose()
        cols_to_drop = ['BALANCE SHEET', 'CASH FLOW STATEMENT', 'CHANGES TO SHAREHOLDER EQUITY', 'Ratios']
        df.drop(cols_to_drop, axis=1, inplace=True)
        return df

In [ ]:
train_dir = 'data/oil_gas_train/'

df = pd.concat([read_table(os.path.join(train_dir, file)) for file in tqdm(os.listdir(train_dir))])
# df = df[df.columns[df.nunique(dropna=False) > 1]]
df.shape

In [130]:
df[df.columns[df.nunique(dropna=False) > 1]].shape

(2056, 440)

In [132]:
df.columns[df.nunique(dropna=False) > 1]

Index([                             'Revenue',
                                 'Consensus:',
           '% Surprise | % Est. vs Consensus',
                       'Revenue Growth (YoY)',
                            'Cost of Revenue',
                               'Gross Profit',
                               'Gross margin',
                     'Other Operating Income',
                         'Operating Expenses',
                   'Selling, General & Admin',
       ...
                          'EBITDA / Interest',
                  'EBITDA - CAPEX / Interest',
                            'CAPEX / REVENUE',
                             'CAPEX / EBITDA',
                                 'FCF / DEBT',
       'Gross Leverage (Total Debt / EBITDA)',
           'Net Leverage (Net Debt / EBITDA)',
        'Total Liabilities / Tangible Assets',
                   'Cash + AR / Total Assets',
                          'Cash % of Revenue'],
      dtype='object', name='INCOME STATEMENT', l

In [125]:
df['Total Assets']

INCOME STATEMENT,Total Assets,Total Assets
2011-03-31 00:00:00,0,NaN
2011-06-30 00:00:00,0,NaN
2011-09-30 00:00:00,0,NaN
2011-12-31 00:00:00,624.656,NaN
2012-03-31 00:00:00,688.609024,NaN
2012-06-30 00:00:00,791.235008,NaN
2012-09-30 00:00:00,856.284992,NaN
2012-12-31 00:00:00,1684.009984,NaN
2013-03-31 00:00:00,1819.484032,NaN
2013-06-30 00:00:00,2565.843968,NaN


In [121]:
df.columns[df.nunique() <= 1]

Index([                           nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
                                  nan,                            nan,
      

In [117]:
 df.columns[df.nunique() > 1]

Index([                             'Revenue',
                                 'Consensus:',
           '% Surprise | % Est. vs Consensus',
                       'Revenue Growth (YoY)',
                            'Cost of Revenue',
                               'Gross Profit',
                               'Gross margin',
                     'Other Operating Income',
                         'Operating Expenses',
                   'Selling, General & Admin',
       ...
                          'EBITDA / Interest',
                  'EBITDA - CAPEX / Interest',
                            'CAPEX / REVENUE',
                             'CAPEX / EBITDA',
                                 'FCF / DEBT',
       'Gross Leverage (Total Debt / EBITDA)',
           'Net Leverage (Net Debt / EBITDA)',
        'Total Liabilities / Tangible Assets',
                   'Cash + AR / Total Assets',
                          'Cash % of Revenue'],
      dtype='object', name='INCOME STATEMENT', l

In [106]:
df['Research & Development'].unique()

array([0], dtype=object)

In [96]:
df.columns[204]

float

In [83]:
df.iloc[:, 204:205]

INCOME STATEMENT,NaN
2011-03-31,NaN
2011-06-30,NaN
2011-09-30,NaN
2011-12-31,NaN
2012-03-31,NaN
...,...
2019-06-30,NaN
2019-09-30,NaN
2019-12-31,NaN
2020-03-31,NaN


In [64]:
df.isna().sum()

(2056, 329)

INCOME STATEMENT
Revenue                                                  0
Consensus:                                               0
% Surprise | % Est. vs Consensus                         0
Revenue Growth (YoY)                                   442
Cost of Revenue                                          0
Gross Profit                                             0
Gross margin                                             0
Other Operating Income                                   0
Operating Expenses                                       0
Selling, General & Admin                                 0
Research & Development                                   0
Depreciation & Amortization                              0
Prov. For Doubful Accounts                               0
Other Operating Expenses                                 0
Operating Income                                         0
Operating Margin                                         0
Non-Operating Income (Loss)            